# Valuing European-style Swaptions with QL Example

We value a European swaption using Black's model and try to replicate a QL example at http://khandrikacm.blogspot.com/2014/03/european-style-interest-rate-swaption.html

In [14]:
import numpy as np
import matplotlib.pyplot as plt

In [16]:
from financepy.products.rates import *
from financepy.utils import *
from financepy.market.curves import *
from financepy.models.black import Black

## Building a Ibor Discount Curve

In [17]:
valuation_date = Date(28, 2, 2014)
settlement_date = Date(4, 3, 2014)

We have a vector of dates and zero rates

In [18]:
depoDCCType = DayCountTypes.THIRTY_E_360_ISDA
depos = []

depo = IborDeposit(settlement_date, "1W", 0.0023, depoDCCType); depos.append(depo)
depo = IborDeposit(settlement_date, "1M", 0.0023, depoDCCType); depos.append(depo)
depo = IborDeposit(settlement_date, "3M", 0.0023, depoDCCType); depos.append(depo)
depo = IborDeposit(settlement_date, "6M", 0.0023, depoDCCType); depos.append(depo)

# No convexity correction provided so I omit interest rate futures

swaps = []
swapType = SwapTypes.PAY
fixedDCCType = DayCountTypes.ACT_365F
fixedFreqType = FrequencyTypes.SEMI_ANNUAL

swap = IborSwap(settlement_date, "3Y", swapType, 0.00790, fixedFreqType, fixedDCCType); swaps.append(swap)
swap = IborSwap(settlement_date, "4Y", swapType, 0.01200, fixedFreqType, fixedDCCType); swaps.append(swap)
swap = IborSwap(settlement_date, "5Y", swapType, 0.01570, fixedFreqType, fixedDCCType); swaps.append(swap)
swap = IborSwap(settlement_date, "6Y", swapType, 0.01865, fixedFreqType, fixedDCCType); swaps.append(swap)
swap = IborSwap(settlement_date, "7Y", swapType, 0.02160, fixedFreqType, fixedDCCType); swaps.append(swap)
swap = IborSwap(settlement_date, "8Y", swapType, 0.02350, fixedFreqType, fixedDCCType); swaps.append(swap)
swap = IborSwap(settlement_date, "9Y", swapType, 0.02540, fixedFreqType, fixedDCCType); swaps.append(swap)
swap = IborSwap(settlement_date, "10Y", swapType, 0.0273, fixedFreqType, fixedDCCType); swaps.append(swap)
swap = IborSwap(settlement_date, "15Y", swapType, 0.0297, fixedFreqType, fixedDCCType); swaps.append(swap)
swap = IborSwap(settlement_date, "20Y", swapType, 0.0316, fixedFreqType, fixedDCCType); swaps.append(swap)
swap = IborSwap(settlement_date, "25Y", swapType, 0.0335, fixedFreqType, fixedDCCType); swaps.append(swap)
swap = IborSwap(settlement_date, "30Y", swapType, 0.0354, fixedFreqType, fixedDCCType); swaps.append(swap)

libor_curve = IborSingleCurve(valuation_date, depos, [], swaps)

In [19]:
exercise_date = settlement_date.add_tenor("5Y")
swapMaturityDate = exercise_date.add_tenor("5Y")

swapFixedCoupon = 0.040852
swapFixedFrequencyType = FrequencyTypes.SEMI_ANNUAL
swapFixedDayCountType = DayCountTypes.THIRTY_E_360_ISDA

swapFloatFrequencyType = FrequencyTypes.QUARTERLY
swapFloatDayCountType = DayCountTypes.ACT_360

swapNotional = ONE_MILLION
swapType = SwapTypes.PAY 
calendar_type = CalendarTypes.TARGET
bus_day_adjust_type = BusDayAdjustTypes.MODIFIED_FOLLOWING
date_gen_rule_type = DateGenRuleTypes.BACKWARD

In [20]:
swaption = IborSwaption(settlement_date,
                            exercise_date,
                            swapMaturityDate,
                            swapType,
                            swapFixedCoupon,
                            swapFixedFrequencyType,
                            swapFixedDayCountType, 
                            swapNotional, 
                            swapFloatFrequencyType,
                            swapFloatDayCountType,
                            calendar_type, 
                            bus_day_adjust_type,
                            date_gen_rule_type)

In [21]:
print(swaption)

OBJECT TYPE: IborSwaption
SETTLEMENT DATE: 04-MAR-2014
EXERCISE DATE: 04-MAR-2019
SWAP FIXED LEG TYPE: SwapTypes.PAY
SWAP MATURITY DATE: 04-MAR-2024
SWAP NOTIONAL: 1000000
FIXED COUPON: 4.0852
FIXED FREQUENCY: FrequencyTypes.SEMI_ANNUAL
FIXED DAY COUNT: DayCountTypes.THIRTY_E_360_ISDA
FLOAT FREQUENCY: FrequencyTypes.QUARTERLY
FLOAT DAY COUNT: DayCountTypes.ACT_360



In [22]:
model = Black(0.1533)

In [23]:
swaption.value(settlement_date, libor_curve, model)

23177.738747208212

This is close to a value of 23162 using QL

## Internals

In [24]:
print(swaption)

OBJECT TYPE: IborSwaption
SETTLEMENT DATE: 04-MAR-2014
EXERCISE DATE: 04-MAR-2019
SWAP FIXED LEG TYPE: SwapTypes.PAY
SWAP MATURITY DATE: 04-MAR-2024
SWAP NOTIONAL: 1000000
FIXED COUPON: 4.0852
FIXED FREQUENCY: FrequencyTypes.SEMI_ANNUAL
FIXED DAY COUNT: DayCountTypes.THIRTY_E_360_ISDA
FLOAT FREQUENCY: FrequencyTypes.QUARTERLY
FLOAT DAY COUNT: DayCountTypes.ACT_360
PV01: 4.160180810139469
FWD SWAP RATE: 4.087007644320825
FWD DF TO EXPIRY: 0.923742195746149


We can see that the forward swap rate almost equals the fixed coupon. The underlying swap is close to being ATM forward.

In [25]:
swaption.print_swap_fixed_leg()

START DATE: 04-MAR-2019
MATURITY DATE: 04-MAR-2024
COUPON (%): 4.0852
FREQUENCY: FrequencyTypes.SEMI_ANNUAL
DAY COUNT: DayCountTypes.THIRTY_E_360_ISDA
PAY_DATE     ACCR_START   ACCR_END     DAYS  YEARFRAC    RATE      PAYMENT       DF          PV        CUM PV
04-SEP-2019  04-MAR-2019  04-SEP-2019   180  0.500000   4.08520     20426.00  0.90796633     18546.12     18546.12
04-MAR-2020  04-SEP-2019  04-MAR-2020   180  0.500000   4.08520     20426.00  0.89260443     18232.34     36778.46
04-SEP-2020  04-MAR-2020  04-SEP-2020   180  0.500000   4.08520     20426.00  0.87443197     17861.15     54639.61
04-MAR-2021  04-SEP-2020  04-MAR-2021   180  0.500000   4.08520     20426.00  0.85691681     17503.38     72142.99
06-SEP-2021  04-MAR-2021  06-SEP-2021   182  0.505556   4.08520     20652.96  0.84037737     17356.28     89499.26
04-MAR-2022  06-SEP-2021  04-MAR-2022   178  0.494444   4.08520     20199.04  0.82476188     16659.40    106158.67
05-SEP-2022  04-MAR-2022  05-SEP-2022   181  0.50

In [26]:
swaption.print_swap_float_leg()

START DATE: 04-MAR-2019
MATURITY DATE: 04-MAR-2024
SPREAD (BPS): 0.0
FREQUENCY: FrequencyTypes.QUARTERLY
DAY COUNT: DayCountTypes.ACT_360
Payments not calculated.


Copyright (c) 2020 Dominic O'Kane